In [ ]:
# Given 8 players, each can put forwards one ancestral weapon.
# Four weapons with the highest stats determine the bonus attributes.
# Each player has a stash of weapons, how we can determine the best combination?
# Weapons have 5 attributes and the the highest 4 weapons need to have balannced stats (ie. the best combination is determined by the maximum of the minimum attributes in sets)

from csv import DictReader

# reading the values from a csv file
# file should be format as follows:
# player,str,int,will,fort,vit, - headers that will not be used
# player1,100,0,100,0,0         - weapon stats and the players who owns the weapon

with open('warband.csv','r', encoding='utf-8-sig') as file:
    
    dict_reader = DictReader(file)
    
    header = []
    player_data={}
    index = {}
    
    for row in dict_reader:
        value_lst = []
        sub_d={}
        
        value_lst.append(int(row['str']))
        value_lst.append(int(row['int']))
        value_lst.append(int(row['will']))
        value_lst.append(int(row['fort']))
        value_lst.append(int(row['vit']))
        
        # no key names - automatically generates       
        if row['player'] not in header:
            index[row['player']]=1
            key = 'w'+str(index[row['player']])+str(row['player'])
            index[row['player']]+=1
            
            sub_d[key]=value_lst
            player_data[row['player']]=sub_d
            
            header.append(row['player'])
        else:
            key = 'w'+str(index[row['player']])+str(row['player'])
            index[row['player']]+=1
            
            sub_d[key]=value_lst
            player_data[row['player']].update(sub_d)
            
file.close()

# we need players name as a list in order to iterate using an index in a recursive function
players = list(player_data.keys())

#function _check_if_worse can check if a weapon from a player is worse than everything else and can be scraped instead of holding in inventory
def check_if_worse(player,weapon):
    weapons = player_data[player]
    stat1_lower = 0
    stat2_lower = 0
    stat3_lower = 0
    stat1_eq = 0
    stat2_eq = 0
    stat3_eq = 0
    
    for w in weapons.values():
        if (w == weapon):
            continue
        stat1_lower = 1
        stat2_lower = 1
        stat3_lower = 1
        
        stat1_eq = 0
        stat2_eq = 0
        stat3_eq = 0
        
        index = 0
        for i in range(0,len(weapon)):
            if weapon[i] != 0:
                index +=1
                if index == 1:
                    if weapon[i]>=w[i]:
                        stat1_lower = 0
                    if weapon[i]==w[i]:
                        stat1_eq = 1
                elif index == 2:
                    if weapon[i]>=w[i]:
                        stat2_lower = 0
                    if weapon[i]==w[i]:
                        stat2_eq = 1
                elif index == 3:
                    if weapon[i]>=w[i]:
                        stat2_lower = 0
                    if weapon[i]==w[i]:
                        stat2_eq = 1
                    
                
                
        if (stat1_lower and stat2_lower and stat3_lower):
            return True
        if (stat1_eq and stat2_lower and stat3_lower):
            return True
        if (stat2_eq and stat1_lower and stat3_lower):
            return True
        if (stat3_eq and stat1_lower and stat2_lower):
            return True
        if (stat1_eq and stat2_eq and stat3_lower):
            return True
        if (stat2_eq and stat3_eq and stat1_lower):
            return True
        if (stat1_eq and stat3_eq and stat3_lower):
            return True
        
    return False
                
weapon_set=[]
all_weapon_set =[]

# function to generate all possible combinations of weapons in the ancestral tableau
def GenerateSets(player_data,index):
    global all_weapon_set
    #print(len(weapon_set),weapon_set,all_weapon_set)
    
    if (len(weapon_set)==8):
        all_weapon_set.append(list(weapon_set))
        #print(all_weapon_set)
        return 
    
    for w_name, w_values in player_data[players[index]].items():
        #weapon_set.append(str(players[index])+w_name) #used this to check if only one item from each p, but I need vaalues
        weapon_set.append(w_values)
        GenerateSets(player_data, index+1)
        weapon_set.pop() 

#parse the data and clean it
to_remove = []
for player, weapons in player_data.items():  
    for w_key, w in weapons.items():
        if check_if_worse(player,w):
            to_remove.append([player,w_key])

#create a new dict for cleaned data. This will be used in further calcualtions
cleaned_player_data = dict(player_data)
for item in to_remove:
    print(item[0], item[1], cleaned_player_data[item[0]][item[1]],' can be scraped.')
    del cleaned_player_data[item[0]][item[1]]

# generate the sets in all_weapon_set
GenerateSets(cleaned_player_data,0)

# too many nested list gets confusing so I break down the process in small functions.
# get the total stats for a weapon (a weapon = a list of attributes)
def stats(weapon):
    return sum(weapon)


# sort index return indexed of a sorted list, default descending
def sort_index(lst, rev=True):
    index = range(len(lst))
    s = sorted(index, reverse=rev, key=lambda i: lst[i])
    return s

# having a list of lists, find the index of the 4 inner lists with the max stats
# max_stats returns a list of values which correspons to the total stas of each weapon
def max_stats(weapons):
    stats_list = []
    for weapon in weapons:
        stats_list.append(stats(weapon))
    #return sort_index(stats_list)[:4]
    return stats_list

# returns a list of sum of attributes given a list of weapons (each with 5 attributes)- takes only 4 highest ones
def sum_attributes(weapons):
    highest_index = sort_index(max_stats(weapons))[:4]
    st,inte,wil,fort,vit = 0,0,0,0,0
    for i in highest_index:
        st+=weapons[i][0]
        inte+=weapons[i][1]
        wil+=weapons[i][2]
        fort+=weapons[i][3]
        vit+=weapons[i][4]
    return [st,inte,wil,fort,vit]

# finally we apply all these functions to the sets of weapons in all_weapon_sets
list_attributes = []
for w_set in all_weapon_set:
    list_attributes.append(sum_attributes(w_set))
list_attributes

# to determine the best weapon set, we need to determine the maximum from the minimum of each nested list, as the minimum of a weapon in set determines the max potential of that set
# index_min_stat is the index of the best set from all_weapon_set
index_min_stat = list_attributes.index(max(list_attributes,key=min)) #is the maximum between minimum in inner lists

#the best set of items is printed like this, but we'll put this in a function
def display_best(n_best):
    final_index_4 = sort_index(max_stats(all_weapon_set[n_best]))[:4]    
    for i in final_index_4:
        print(players[i]+ ': '+ str(all_weapon_set[n_best][i]))
    
    #list_attributes divided by 4 (integer result) represents the value of maximum bonus in game
    print('...WITH...')
    print("Final values in tableau:\n  STR  INT  WILL FORT VIT\n",[it//4 for it in list_attributes[n_best]])

#See at the end for more than one index
# print('...............................')
# print('BEST Set of Ancestral Weapon is:')
# display_best(index_min_stat)
    
#to determine other best sets:
#for some reason, sets are repeated in the final result, so we need to remove duplicates:
list_attributes_sorted = list(list_attributes)
list_attributes_sorted.sort(key=min, reverse= True)
list_attributes_sorted

unique_set=[]

for item in list_attributes_sorted:
    if item not in unique_set:
        unique_set.append(list(item))

#we just need to find the index from list_attributes, and use this index in our sets of weapons
best=list_attributes.index(unique_set[0])
second_best = list_attributes.index(unique_set[1])
third_best = list_attributes.index(unique_set[2])
fourth = list_attributes.index(unique_set[3])
fifth = list_attributes.index(unique_set[4])

print('...............................')
print('BEST Set of Ancestral Weapon is:')
display_best(best)

print('...............................')
print('2ND Best Set of Ancestral Weapon is:')
display_best(second_best)

print('...............................')
print('3RD Best Set of Ancestral Weapon is:')
display_best(third_best)

print('...............................')
print('4TH Best Set of Ancestral Weapon is:')
display_best(fourth)

print('...............................')
print('5TH Best Set of Ancestral Weapon is:')
display_best(fifth)
